In [7]:
import sys

sys.path.extend(['.', '..'])

import platform
import os
import numpy as np
import tensorflow as tf
import keras
from keras import layers
import pandas as pd

gpu_token = tf.test.is_gpu_available('GPU')

if platform.machine() == 'x86_64':
    if gpu_token:
        os.chdir('/home/ucdasec/Faulthunter-RNN-approach/manual_dataset')
    else:
        os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/manual_dataset")
elif platform.machine() == 'arm64':
    os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/dataset")

from pycparser import c_parser, c_generator
import pycparser
from Utils.w2b_encoding import *
from Utils.cparser_dataframe_gen import *

text=[]
counter = 0
with open('TrustFlex pub_key_rotate.c') as obj:
    lines = obj.readlines()
    for line in lines:
        if line.__contains__("#"):
                pass
        else:
            text.append(line)


    # temp = obj.read()
    # obj.seek(0)
    # while obj.tell()!=len(temp):
    #     print(counter)
    #     counter+=1
    #     try:
    #         line = obj.readline()
    #         if line.__contains__("#"):
    #             pass
    #         else:
    #             text.append(line)
    #     except EOFError:
    #         break


text = "".join(text)
print(text)


/** \file
 * \brief Public Key Rotation
 *
 * \copyright (c) 2015-2018 Microchip Technology Inc. and its subsidiaries.
 *
 * \page License
 *
 * Subject to your compliance with these terms, you may use Microchip software
 * and any derivatives exclusively with Microchip products. It is your
 * responsibility to comply with third party license terms applicable to your
 * use of third party software (including open source software) that may
 * accompany Microchip software.
 *
 * THIS SOFTWARE IS SUPPLIED BY MICROCHIP "AS IS". NO WARRANTIES, WHETHER
 * EXPRESS, IMPLIED OR STATUTORY, APPLY TO THIS SOFTWARE, INCLUDING ANY IMPLIED
 * WARRANTIES OF NON-INFRINGEMENT, MERCHANTABILITY, AND FITNESS FOR A
 * PARTICULAR PURPOSE. IN NO EVENT WILL MICROCHIP BE LIABLE FOR ANY INDIRECT,
 * SPECIAL, PUNITIVE, INCIDENTAL OR CONSEQUENTIAL LOSS, DAMAGE, COST OR EXPENSE
 * OF ANY KIND WHATSOEVER RELATED TO THE SOFTWARE, HOWEVER CAUSED, EVEN IF
 * MICROCHIP HAS BEEN ADVISED OF THE POSSIBILITY OR THE DAMAGES 

2024-04-01 14:51:24.887015: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-01 14:51:24.887036: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
df_dictionary = dataframe_init(gpu_token)

In [11]:
df_dictionary['rauc crypt.c']

,File,Line Number,Lines,Label
0,rauc crypt.c,0,#include <errno.h>,Secure
1,rauc crypt.c,1,#include <glib.h>,Secure
2,rauc crypt.c,2,#include <glib/gstdio.h>,Secure
3,rauc crypt.c,3,#include <openssl/evp.h>,Secure
4,rauc crypt.c,4,,Secure
...,...,...,...,...
146,rauc crypt.c,146,,Secure
147,rauc crypt.c,147,"gboolean r_crypt_decrypt(const gchar *in, cons...",Secure
148,rauc crypt.c,148,{,Secure
149,rauc crypt.c,149,"return r_crypt_encrypt_or_decrypt(in, out, key...",Secure


In [8]:
parser = c_parser.CParser()
generator = c_generator.CGenerator()
ast = parser.parse(text, filename='??.c')
explore = {pycparser.c_ast.If: {'node': (),'key': 'cond', 'branches': ('iftrue', 'iffalse')}, 
                       pycparser.c_ast.FuncCall: {'node': (),'key' : 'name' , 'branches': ()},
                       pycparser.c_ast.FuncDef: {'node': (),'key' : 'decl', 'branches': ('body',)},
                       pycparser.c_ast.FileAST: {'node': (),'key' : 'ext[0]', 'branches': ()}
                       } 
                    #    pycparser.c_ast.Switch: (),
                    #    pycparser.c_ast.For: ()}

If_explore = {pycparser.c_ast.BinaryOp : {'branches': ('left', 'right')}}

HAMMING_WEIGHT = 4

In [9]:
parent_ast = ast#.children()[0][1].body

NODE_LIST = []

# def Func_walk(func_node):
#     if func_node.children():
#         sub_node = func_node.children()
#         for node in sub_node:
            

def walk(parent_ast, flag):
    if parent_ast.children():
        sub_ast = parent_ast.children()
        # if sub_ast[0][1].decl.name == 'main' and not(flag):
        #     walk(sub_ast[0][1], flag = 1)
        for node in sub_ast:
            if type(node[1]) in list(explore.keys()):
                node_attrs = list((explore[type(node[1])]['key'],) + explore[type(node[1])]['branches'])
                node_attr = {i: False for i in node_attrs}
                for sub_node in node[1].children():
                    node_attr[sub_node[0]] = sub_node
                node_attr['node'] = ('node', node[1],)

                print(type(node[1]), end = " : ")
                print(node_attr[explore[type(node[1])]['key']])
                NODE_LIST.append([('type',type(node_attr['node'][1])), node_attr[explore[type(node[1])]['key']], node_attr['node']])
                for sub_node in explore[type(node[1])]['branches']:
                    print(sub_node, " : {}".format(type(node_attr[sub_node][1].block_items[0]) if node_attr[sub_node] else node_attr[sub_node]))
                    if  node_attr[sub_node]:
                        walk(node_attr[sub_node][1], 1)
            # elif (type(node[1]) == pycparser.c_ast.FuncCall):  

            else:
                print(type(node[1]))
                NODE_LIST.append([('type', type(node[1])), ('key', False), ('node', node[1])])
        
          
walk(parent_ast, 0)



<class 'pycparser.c_ast.FuncDef'> : ('decl', Decl(name='main',
     quals=[
           ],
     align=[
           ],
     storage=[
             ],
     funcspec=[
              ],
     type=FuncDecl(args=ParamList(params=[Typename(name=None,
                                                   quals=[
                                                         ],
                                                   align=None,
                                                   type=TypeDecl(declname=None,
                                                                 quals=[
                                                                       ],
                                                                 align=None,
                                                                 type=IdentifierType(names=['void'
                                                                                           ]
                                                                               

In [10]:
START = 0
for i in NODE_LIST:
    if i[1][1]:
        line = ((generator.visit(i[1][1])))
    else:
        line = ((generator.visit(i[2][1])))
    start = text.find(line, START)
    end = len(line) + start
    i.append((start,end))
    START=end

NODE_LIST

[[('type', pycparser.c_ast.FuncDef),
  ('decl', Decl(name='main',
        quals=[
              ],
        align=[
              ],
        storage=[
                ],
        funcspec=[
                 ],
        type=FuncDecl(args=ParamList(params=[Typename(name=None,
                                                      quals=[
                                                            ],
                                                      align=None,
                                                      type=TypeDecl(declname=None,
                                                                    quals=[
                                                                          ],
                                                                    align=None,
                                                                    type=IdentifierType(names=['void'
                                                                                              ]
                      

In [11]:
If_explore = {pycparser.c_ast.BinaryOp : {'branches': (pycparser.c_ast.BinaryOp.left, pycparser.c_ast.BinaryOp.right)}}

def if_walk(if_node):
    if type(if_node) in If_explore.keys():
        kids = if_node.children()
        kids = [type(i[1]) for i in kids]
        if list(If_explore.keys())[0] in kids:
            for child in if_node.children():
                if_walk(child[1])
        else:
            SUB_NODE_LIST.append(if_node)
            return
                  

    elif type(if_node) in If_explore[type(if_node)]['branches']:

        SUB_NODE_LIST.append(if_node)

modify_list = []
for i in enumerate(NODE_LIST):
    SUB_NODE_LIST = []
    sub_row = []
    if i[1][0][1] == pycparser.c_ast.If:
        cond = i[1][1][1]
        if_walk(cond)
        for cond in SUB_NODE_LIST:
            temp = i[1].copy()
            temp[1] = ('cond', cond)
            sub_row.append(temp)
        modify_list.append((i[0], sub_row))

modify_list
for mod in modify_list[::-1]:
    NODE_LIST.pop(mod[0])
    for items in mod[1][::-1]:
        NODE_LIST.insert(mod[0], items)

NODE_LIST


        

[[('type', pycparser.c_ast.FuncDef),
  ('decl', Decl(name='main',
        quals=[
              ],
        align=[
              ],
        storage=[
                ],
        funcspec=[
                 ],
        type=FuncDecl(args=ParamList(params=[Typename(name=None,
                                                      quals=[
                                                            ],
                                                      align=None,
                                                      type=TypeDecl(declname=None,
                                                                    quals=[
                                                                          ],
                                                                    align=None,
                                                                    type=IdentifierType(names=['void'
                                                                                              ]
                      

In [16]:
dataframe = pd.DataFrame(columns=['Type', 'Key Line', 'Line', '(start, end)','Key Node', 'Node', 'Encoded Line'])

for i in NODE_LIST:
    row = {'Type': None, 'Key Line': None, 'Line' : None, '(start, end)' : None, 'Key Node': None, 'Node': None, 'Encoded Line' : None}
    #type
    row['Type'] = i[0][1]

    #key line
    if i[1][1]:
        row['Key Line'] = (generator.visit(i[1][1]))
    else:
        row['Key Line'] = (generator.visit(i[2][1]))

    #line
    row['Line'] = text[i[3][0] : i[3][1]]

    #start, end
    row['(start, end)'] = (i[3])

    #key node
    row['Key Node'] = i[1][1]

    #node
    row['Node'] = i[2]
    dataframe.loc[len(dataframe)] = row


    

In [17]:
dataframe.head(10)

,Type,Key Line,Line,"(start, end)",Key Node,Node,Encoded Line
0,<class 'pycparser.c_ast.FuncDef'>,int main(void),int main(void),"(0, 14)","Decl(name='main',\n quals=[\n ],...","(node, FuncDef(decl=Decl(name='main',\n ...",None
1,<class 'pycparser.c_ast.Decl'>,int result = 1,int result = 1,"(21, 35)",False,"(node, Decl(name='result',\n quals=[\n ...",None
2,<class 'pycparser.c_ast.If'>,result > 1,(result > 1) || (result < 3),"(44, 72)","BinaryOp(op='>',\n left=ID(name='resul...","(node, If(cond=BinaryOp(op='||',\n ...",None
3,<class 'pycparser.c_ast.If'>,result < 3,(result > 1) || (result < 3),"(44, 72)","BinaryOp(op='<',\n left=ID(name='resul...","(node, If(cond=BinaryOp(op='||',\n ...",None
4,<class 'pycparser.c_ast.If'>,result < 3,result < 3,"(104, 114)","BinaryOp(op='<',\n left=ID(name='resul...","(node, If(cond=BinaryOp(op='<',\n ...",None
5,<class 'pycparser.c_ast.FuncCall'>,printf,printf,"(129, 135)",ID(name='printf'\n ),"(node, FuncCall(name=ID(name='printf'\n ...",None
6,<class 'pycparser.c_ast.FuncCall'>,printf,printf,"(213, 219)",ID(name='printf'\n ),"(node, FuncCall(name=ID(name='printf'\n ...",None
7,<class 'pycparser.c_ast.Return'>,return 1;,return 1;,"(248, 257)",False,"(node, Return(expr=Constant(type='int',\n ...",None
8,<class 'pycparser.c_ast.Return'>,return 0;,return 0;,"(292, 301)",False,"(node, Return(expr=Constant(type='int',\n ...",None


In [26]:
dataframe['Key Node'][2].right

Constant(type='int',
         value='1'
         )

In [ ]:
If_explore = {pycparser.c_ast.BinaryOp : {'key':(), 'branches': ('left', 'right')}}


for i in dataframe.iloc():
    line = []
    if i.Type == pycparser.c_ast.If:
        line.append('0')
        node = i.Node[1]
        if type(node.cond.left) == pycparser.c_ast.Constant:
            value = int(node.cond.left.value)
        elif type(node.cond.right) == pycparser.c_ast.Constant:
            value = int(node.cond.right.value)
        else:
            print("Error")
        
        value = str(bin(value))[2:]
        line.extend(list('0'*(256 - len(value)) + value))

    else:
        line.append('1')
        line.extend(list('1'*256))

    line = [eval(i) for i in line]
    i['Encoded Line'] = line



In [13]:
dir(pycparser.c_ast.BinaryOp)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'attr_names',
 'children',
 'coord',
 'left',
 'op',
 'right',
 'show']